In [1]:
# Library Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.metrics import mean_squared_error
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import KFold  # Assuming no stratification needed for regression
import gc
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import log_evaluation, early_stopping
import re
import polars as pl
from catboost import CatBoostRegressor, Pool
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_squared_error

pd.set_option('future.no_silent_downcasting', True)

In [2]:
train_data=pl.read_csv("/kaggle/input/brist1d/train.csv")
train_data=train_data.to_pandas()
test_data=pl.read_csv("/kaggle/input/brist1d/test.csv")
test_data=test_data.to_pandas()
print(len(train_data),len(test_data))
train_data.head()

177024 3644


,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,None,NaN,9.6,None,NaN,9.7,None,...,None,None,None,None,None,None,None,None,None,13.4
1,p01_1,p01,06:25:00,None,NaN,9.7,None,NaN,9.2,None,...,None,None,None,None,None,None,None,None,None,12.8
2,p01_2,p01,06:40:00,None,NaN,9.2,None,NaN,8.7,None,...,None,None,None,None,None,None,None,None,None,15.5
3,p01_3,p01,06:55:00,None,NaN,8.7,None,NaN,8.4,None,...,None,None,None,None,None,None,None,None,None,14.8
4,p01_4,p01,07:10:00,None,NaN,8.4,None,NaN,8.1,None,...,None,None,None,None,None,None,None,None,None,12.7


In [3]:
test_id = test_data["id"]

In [4]:
#six float features
feats=['bg','insulin','hr','steps','cals','activity']
ts=['-5:55', '-5:50', '-5:45', '-5:40', '-5:35', '-5:30', '-5:25', '-5:20', 
    '-5:15', '-5:10', '-5:05', '-5:00', '-4:55', '-4:50', '-4:45', '-4:40', 
    '-4:35', '-4:30', '-4:25', '-4:20', '-4:15', '-4:10', '-4:05', '-4:00', 
    '-3:55', '-3:50', '-3:45', '-3:40', '-3:35', '-3:30', '-3:25', '-3:20', 
    '-3:15', '-3:10', '-3:05', '-3:00', '-2:55', '-2:50', '-2:45', '-2:40', 
    '-2:35', '-2:30', '-2:25', '-2:20', '-2:15', '-2:10', '-2:05', '-2:00', 
    '-1:55', '-1:50', '-1:45', '-1:40', '-1:35', '-1:30', '-1:25', '-1:20', 
    '-1:15', '-1:10', '-1:05', '-1:00', '-0:55', '-0:50', '-0:45', '-0:40', 
    '-0:35', '-0:30', '-0:25', '-0:20', '-0:15', '-0:10', '-0:05', '-0:00']
def get_more_data(df,timestep,mode='train'):#mode is train or test
    full_data=[]
    #12step is 1 hour.
    for start in range(0,len(ts)-timestep-12):
        end=start+timestep
        target=end+12
        #[start:end] target
        hours_data=df[['p_num']+[f+ts[i]  for i in range(start,end) for f in feats]+['bg'+ts[target]]]
        hours_data.columns=['p_num']+[f+str(i-start)  for i in range(start,end)for f in feats]+['bg+1:00']
        hours_data=hours_data[~hours_data['bg+1:00'].isna()]
        full_data.append(hours_data)
    if mode=='test':
        hours_data=df[['p_num']+[f+ts[i]  for i in range(len(ts)-timestep,len(ts)) for f in feats]]
        hours_data.columns=['p_num']+[f+str(i-len(ts)+timestep)  for i in range(len(ts)-timestep,len(ts))for f in feats]
        full_data.append(hours_data)
    full_data=pd.concat(full_data).drop_duplicates()
    return full_data

In [5]:
def FE(df, mode='train', timestep=36):
    print("< drop useless columns >")
    df.drop([c for c in df.columns if 'carbs' in c] + ['id'], axis=1, inplace=True)

    print("< fillna >")
    bgs = [c for c in list(df.columns) if (c != 'bg+1:00') and ('bg' in c)]
    df[bgs] = df[bgs].ffill(axis=1)
    df[bgs] = df[bgs].bfill(axis=1)

    print("< log1p transform >")
    step_cols = [c for c in list(df.columns) if 'step' in c]
    df[step_cols] = df[step_cols].fillna(50).astype(np.float32)
    df[step_cols] = np.log1p(df[step_cols])
    
    print("< fix bug >")
    float_cols = [c for c in df.columns for f in feats[:-1] if f in c]
    df[float_cols] = df[float_cols].astype(np.float32)

    print("< get more data >")
    df = get_more_data(df, timestep=timestep, mode=mode)

    df['p_num_int'] = df['p_num'].apply(lambda x: int(x[1:])).astype(np.int8)

    print("< fill missing feature values >")
    float_cols = [col + str(ts) for ts in range(timestep) for col in ['bg', 'insulin', 'hr', 'steps', 'cals']]
    df[float_cols] = df[float_cols].fillna(-1)

    print("< activity feature >")
    activity_cols = [c for c in list(df.columns) if 'activity' in c]
    activity_feats = df[activity_cols].replace('None', '').fillna('')
    df['activity_present'] = activity_feats.apply(lambda row: any(row), axis=1).astype(int)
    df.drop(activity_cols, axis=1, inplace=True)

    print("< Z-score feature >")
    cols = ['bg', 'insulin', 'steps', 'hr', 'cals']

    # Create columns for timestep suffix only once
    suffix = f"{timestep - 1}"
    cols_with_suffix = [f"{col}{suffix}" for col in cols]

    # Group by `p_num` and calculate mean and std for each feature at the given timestep
    grouped_stats = df.groupby('p_num')[cols_with_suffix].agg(['mean', 'std']).reset_index()
    grouped_stats.columns = ['p_num'] + [f"{stat}_pnum_{col}" for col in cols_with_suffix for stat in ['mean', 'std']]

    # Join the mean and std columns back to the original DataFrame on `p_num`
    df = df.merge(grouped_stats, on='p_num', how='left')

    # Calculate z-scores for each column using the joined mean and std
    z_score_features = {}
    for col in cols:
        mean_col = f'mean_pnum_{col}{suffix}'
        std_col = f'std_pnum_{col}{suffix}'
        z_score_features[f'norm_pnum_{col}{suffix}'] = (df[f'{col}{suffix}'] - df[mean_col]) / df[std_col]

    # Convert z-score calculations to DataFrame and join all at once to avoid fragmentation
    z_score_df = pd.DataFrame(z_score_features)
    df = pd.concat([df, z_score_df], axis=1)

    print("< cross feature >")
    cross = ['bg', 'insulin', 'steps', 'hr', 'cals']
    for i in range(len(cross)):
        for j in range(i + 1, len(cross)):
            for k in [1]:
                df[f"{cross[i]}{timestep - k}_{cross[j]}{timestep - k}"] = df[f"{cross[i]}{timestep - k}"] * df[f"{cross[j]}{timestep - k}"]

    return df



In [6]:
train = FE(train_data, mode='train')
test = FE(test_data, mode='test')
total = pd.concat((train, test))
test = total[total['bg+1:00'].isna()]
train = total[~total['bg+1:00'].isna()]
test.drop(["bg+1:00"], axis=1, inplace=True)

print(len(train), len(test), len(total))
train.head()

< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >
< fill missing feature values >
< activity feature >


/tmp/ipykernel_23/581369802.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['activity_present'] = activity_feats.apply(lambda row: any(row), axis=1).astype(int)


< Z-score feature >
< cross feature >
< drop useless columns >
< fillna >
< log1p transform >
< fix bug >
< get more data >


/tmp/ipykernel_23/581369802.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['p_num_int'] = df['p_num'].apply(lambda x: int(x[1:])).astype(np.int8)


< fill missing feature values >
< activity feature >


/tmp/ipykernel_23/581369802.py:31: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['activity_present'] = activity_feats.apply(lambda row: any(row), axis=1).astype(int)


< Z-score feature >
< cross feature >
356075 3644 359719


/tmp/ipykernel_23/2553637795.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(["bg+1:00"], axis=1, inplace=True)


,p_num,bg0,insulin0,hr0,steps0,cals0,bg1,insulin1,hr1,steps1,...,bg35_insulin35,bg35_steps35,bg35_hr35,bg35_cals35,insulin35_steps35,insulin35_hr35,insulin35_cals35,steps35_hr35,steps35_cals35,hr35_cals35
0,p01,9.6,0.0083,-1.0,3.931826,-1.0,9.6,0.0083,-1.0,3.931826,...,0.11537,54.652374,-13.900000,-13.900000,0.032634,-0.0083,-0.0083,-3.931826,-3.931826,1.0
1,p01,9.7,0.0083,-1.0,3.931826,-1.0,9.7,0.0083,-1.0,3.931826,...,0.11786,55.831924,-14.200000,-14.200000,0.032634,-0.0083,-0.0083,-3.931826,-3.931826,1.0
2,p01,9.2,0.0083,-1.0,3.931826,-1.0,9.2,0.0083,-1.0,3.931826,...,0.11786,55.831924,-14.200000,-14.200000,0.032634,-0.0083,-0.0083,-3.931826,-3.931826,1.0
3,p01,8.7,0.0083,-1.0,3.931826,-1.0,8.7,0.0083,-1.0,3.931826,...,0.12782,60.550114,-15.400000,-15.400000,0.032634,-0.0083,-0.0083,-3.931826,-3.931826,1.0
4,p01,8.4,0.0083,-1.0,3.931826,-1.0,8.4,0.0083,-1.0,3.931826,...,1.00276,67.627403,-17.200001,-17.200001,0.229225,-0.0583,-0.0583,-3.931826,-3.931826,1.0


In [7]:
def cat_process(data, timestep=36):
    df = data.copy()
    cols = ['bg', 'insulin', 'steps', 'hr', 'cals']
    suffix = f"{timestep - 1}"

    for col in cols:
        df[f'mean_pnum_{col}{suffix}'] = df[f'mean_pnum_{col}{suffix}'].astype('string').astype('category')
        df[f'std_pnum_{col}{suffix}'] = df[f'std_pnum_{col}{suffix}'].astype('string').astype('category')
        df[f'norm_pnum_{col}{suffix}'] = df[f'norm_pnum_{col}{suffix}'].astype('string').astype('category')
        
    df['p_num'] = df['p_num'].astype('string').astype('category')
    df['activity_present'] = df['activity_present'].astype('string').astype('category')
    
    return df
        

In [8]:
train_cat = cat_process(train)
test_cat = cat_process(test)

In [9]:
train_cat.drop(["p_num_int"], axis=1, inplace=True)
test_cat.drop(["p_num_int"], axis=1, inplace=True)

train.drop(["p_num"], axis=1, inplace=True)
test.drop(["p_num"], axis=1, inplace=True)

/tmp/ipykernel_23/1964283331.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(["p_num"], axis=1, inplace=True)
/tmp/ipykernel_23/1964283331.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop(["p_num"], axis=1, inplace=True)


In [10]:
def cross_validate_cat(params, X, y, test_data, n_splits=2, n_bags=1, random_state=42):
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(test_data))
    scores = []
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    cat_features = X.select_dtypes(include=['category', 'object']).columns.tolist()
    
    for fold, (train_index, valid_index) in enumerate(kfold.split(X, y)):
        for i in range(n_bags):
            
            # Split train and validation data
            X_train = X.iloc[train_index].copy()
            y_train = y.iloc[train_index]
            X_val = X.iloc[valid_index].copy()
            y_val = y.iloc[valid_index]
            
            # Convert categorical features to strings
            for cat_col in cat_features:
                X_train[cat_col] = X_train[cat_col].astype(str)
                X_val[cat_col] = X_val[cat_col].astype(str)
                test_data[cat_col] = test_data[cat_col].astype(str)
                
            # Create CatBoost Pools for train, validation, and test data
            train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
            val_pool = Pool(data=X_val, label=y_val, cat_features=cat_features)
            test_pool = Pool(data=test_data, cat_features=cat_features)

            model = CatBoostRegressor(**params, eval_metric='RMSE', random_seed=42, early_stopping_rounds=100, verbose=100, task_type='GPU')
            # Fit model
            model.fit(train_pool, eval_set=val_pool)
            
            y_pred = model.predict(val_pool)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            print(f"# Fold {fold}, bag {i}: RMSE={rmse:.5f}")
            scores.append(rmse)
            
            oof_preds[valid_index] += y_pred / n_bags
            test_preds += model.predict(test_pool) / kfold.get_n_splits() / n_bags
            
            del model
            gc.collect()
            
        print(f"#RMSE mean: {np.mean(scores):.7f} (+- {np.std(scores):.7f})")

    return oof_preds, test_preds

In [11]:
def cross_validate(params, model_type, X, y, test_data, n_splits=2, n_bags = 1, random_state=42):
    oof_preds = np.zeros(len(X))
    test_preds = np.zeros(len(test_data))
    scores = []
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    for fold, (train_index, valid_index) in enumerate(kfold.split(X,y)):
        for i in range(0,n_bags):
            X_train = X.iloc[train_index]
            y_train = y.iloc[train_index]
            X_val = X.iloc[valid_index]
            y_val = y.iloc[valid_index]
            
            if model_type == 'lgb':
                model = LGBMRegressor(**params)
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)],callbacks = callbacks)
            elif model_type == 'xgb':
                model = XGBRegressor(**params, early_stopping_rounds=100, n_jobs=4, random_state=42, max_bin=100000, device="cuda")
                model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 300)
            
            y_pred = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            print(f"# Fold {fold}, bag {i}: RMSE={rmse:.5f}")
            scores.append(rmse)
            
            oof_preds[valid_index] += y_pred / n_bags
            test_preds += model.predict(test_data) / kfold.get_n_splits() / n_bags
            
            del model
            gc.collect()
            
        print(f"#RMSE mean: {np.mean(scores):.7f} (+- {np.std(scores):.7f})")

    return oof_preds, test_preds

In [12]:
def objective_lgb(trial):
    global lgb_test
    global lgb_oof
    params_lgbm = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 140),
        'learning_rate': trial.suggest_float('learning_rate', 0.15, 0.25),
        'n_estimators': trial.suggest_int('n_estimators', 3000, 4000),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 30),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1),
        'metric': 'root_mean_squared_error',
        'random_state': 42,
        'bagging_freq': 3,
        'device':'gpu',
        'verbose' : -1,
        'n_jobs': 4,
    }
    
    oof_preds_lgb, test_preds_lgb = cross_validate(params_lgbm, "lgb", X, y, test, n_splits = 5, n_bags = 3)

    rmse_score = np.sqrt(mean_squared_error(y, oof_preds_lgb))
    print(f"Trial {trial.number}, final RMSE: {rmse_score}")
    
    if trial.number == 0 or rmse_score < trial.study.best_value:
        lgb_test = test_preds_lgb
        lgb_oof = oof_preds_lgb
        print(f"Saved best test with RMSE score: {rmse_score}")
    
    return rmse_score


In [13]:
def objective_cat(trial):
    global cat_test
    global cat_oof
    params_cat = {
        'iterations': trial.suggest_int('iterations', 1000, 1500),
        'learning_rate': trial.suggest_float('learning_rate', 0.15, 0.25),
        'depth': trial.suggest_int('depth', 10, 15)
    }
    
    oof_preds_cat, test_preds_cat = cross_validate_cat(params_cat, X_cat, y_cat, test_cat, n_splits=5, n_bags=3)

    rmse_score = np.sqrt(mean_squared_error(y_cat, oof_preds_cat))
    print(f"Trial {trial.number}, final RMSE: {rmse_score}")
    
    if trial.number == 0 or rmse_score < trial.study.best_value:
        cat_test = test_preds_cat
        cat_oof = oof_preds_cat
        print(f"Saved best test with RMSE score: {rmse_score}")
    
    return rmse_score

In [14]:
def objective_xgb(trial):
    global xgb_test
    global xgb_oof
    params_xgb = {
        'learning_rate': trial.suggest_float('learning_rate', 0.15, 0.25),
        'n_estimators': trial.suggest_int('n_estimators', 1500, 2500),
        'max_depth': trial.suggest_int('max_depth', 10, 15),
        'reg_alpha': trial.suggest_int('req_alpha', 5, 20),
        'reg_lambda': trial.suggest_int('reg_lambda', 10, 20),
        'min_child_weight': trial.suggest_int('min_child_weight', 15, 25),                                    
        'eval_metric': 'rmse',
    }
    oof_preds_xgb, test_preds_xgb = cross_validate(params_xgb, "xgb", X, y, test, n_splits = 5, n_bags = 1)

    rmse_score = np.sqrt(mean_squared_error(y, oof_preds_xgb))
    print(f"Trial {trial.number}, final RMSE: {rmse_score}")
    
    if trial.number == 0 or rmse_score < trial.study.best_value:
        xgb_test = test_preds_xgb
        xgb_oof = oof_preds_xgb
        print(f"Saved best test with RMSE score: {rmse_score}")
    
    return rmse_score

In [15]:
X = train
y = X.pop('bg+1:00')

X_cat = train_cat
y_cat = X_cat.pop('bg+1:00')

callbacks = [log_evaluation(period=300), early_stopping(stopping_rounds=250)]

In [16]:
# Set up and run the Optuna study
#optuna.logging.set_verbosity(optuna.logging.WARNING)
#study = optuna.create_study(direction="minimize")
#study.optimize(objective_xgb, n_trials=4)
#print("Best parameters:", study.best_params)

In [17]:
best_params = {'learning_rate': 0.24301227753308435, 'n_estimators': 1837, 'max_depth': 12, 'req_alpha': 12, 'reg_lambda': 13, 'min_child_weight': 17, 'eval_metric': 'rmse',}
xgb_oof, xgb_test = cross_validate(best_params, "xgb", X, y, test, n_splits = 5, n_bags = 1)

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [07:35:26] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "req_alpha" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:2.84129
[300]	validation_0-rmse:1.53771
[600]	validation_0-rmse:1.48037
[900]	validation_0-rmse:1.46561
[1200]	validation_0-rmse:1.46147
[1500]	validation_0-rmse:1.45945
[1800]	validation_0-rmse:1.45868
[1836]	validation_0-rmse:1.45861


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [07:49:17] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


# Fold 0, bag 0: RMSE=1.45861
#RMSE mean: 1.4586105 (+- 0.0000000)


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [07:49:23] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "req_alpha" } are not used.

  warnings.warn(smsg, UserWarning)


...

[0]	validation_0-rmse:2.84988
[300]	validation_0-rmse:1.52983
[600]	validation_0-rmse:1.46951
[900]	validation_0-rmse:1.45565
[1200]	validation_0-rmse:1.45091
[1500]	validation_0-rmse:1.44933
[1800]	validation_0-rmse:1.44852
[1836]	validation_0-rmse:1.44846
# Fold 2, bag 0: RMSE=1.44846
#RMSE mean: 1.4499682 (+- 0.0065292)


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:14:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "req_alpha" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:2.85258
[300]	validation_0-rmse:1.51876
[600]	validation_0-rmse:1.46496
[900]	validation_0-rmse:1.45088
[1200]	validation_0-rmse:1.44701
[1500]	validation_0-rmse:1.44498
[1800]	validation_0-rmse:1.44407
[1836]	validation_0-rmse:1.44399
# Fold 3, bag 0: RMSE=1.44399
#RMSE mean: 1.4484737 (+- 0.0062188)


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [08:26:14] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "req_alpha" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-rmse:2.84889
[300]	validation_0-rmse:1.54357
[600]	validation_0-rmse:1.48592
[900]	validation_0-rmse:1.47279
[1200]	validation_0-rmse:1.46850
[1500]	validation_0-rmse:1.46693
[1800]	validation_0-rmse:1.46595
[1836]	validation_0-rmse:1.46588
# Fold 4, bag 0: RMSE=1.46588
#RMSE mean: 1.4519547 (+- 0.0089112)


In [18]:
sample_sub = pd.read_csv("/kaggle/input/brist1d/sample_submission.csv")
sample_sub['bg+1:00'] = xgb_test.astype(np.float16)
sample_sub['id'] = test_id
sample_sub.drop(1)
sample_sub.to_csv('submission_xgb.csv', index=False)
print(sample_sub.head())

         id    bg+1:00
0  p01_8459   8.539062
1  p01_8460   8.312500
2  p01_8461   8.406250
3  p01_8462  11.734375
4  p01_8463   7.371094


In [19]:
# Set up and run the Optuna study
#optuna.logging.set_verbosity(optuna.logging.WARNING)
#study = optuna.create_study(direction="minimize")
#study.optimize(objective_lgb, n_trials=8)
#print("Best parameters:", study.best_params)

In [20]:
best_params = {'num_leaves': 119, 'learning_rate': 0.23201091196307633, 'n_estimators': 3598, 'max_depth': 13, 'min_data_in_leaf': 17, 'feature_fraction': 0.8319694615799249, 'bagging_fraction': 0.7079871043022474, 'metric': 'root_mean_squared_error', 'random_state': 42,
'bagging_freq': 3,
'device':'gpu',
'verbose' : -1,
'n_jobs': 4,}
lgb_oof, lgb_test = cross_validate(best_params, "lgb", X, y, test, n_splits = 5, n_bags = 3)


1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.
1 warning generated.


Training until validation scores don't improve for 250 rounds
[300]	valid_0's rmse: 1.76164
[600]	valid_0's rmse: 1.65978
[900]	valid_0's rmse: 1.60433
[1200]	valid_0's rmse: 1.57003
[1500]	valid_0's rmse: 1.5479
[1800]	valid_0's rmse: 1.53286
[2100]	valid_0's rmse: 1.52205
[2400]	valid_0's rmse: 1.51528
[2700]	valid_0's rmse: 1.50984
[3000]	valid_0's rmse: 1.50624
[3300]	valid_0's rmse: 1.50355
Did not meet early stopping. Best iteration is:
[3598]	valid_0's rmse: 1.5014
# Fold 0, bag 0: RMSE=1.50140
Training until validation scores don't improve for 250 rounds
[300]	valid_0's rmse: 1.75879
[600]	valid_0's rmse: 1.65752
[900]	valid_0's rmse: 1.60222
[1200]	valid_0's rmse: 1.56796
[1500]	valid_0's rmse: 1.54664
[1800]	valid_0's rmse: 1.53218
[2100]	valid_0's rmse: 1.52194
[2400]	valid_0's rmse: 1.51475
[2700]	valid_0's rmse: 1.50939
[3000]	valid_0's rmse: 1.50571
[3300]	valid_0's rmse: 1.50302
Did not meet early stopping. Best iteration is:
[3598]	valid_0's rmse: 1.50096
# Fold 0, bag 

In [21]:
sample_sub = pd.read_csv("/kaggle/input/brist1d/sample_submission.csv")
sample_sub['bg+1:00'] = lgb_test.astype(np.float16)
sample_sub['id'] = test_id
sample_sub.drop(1)
sample_sub.to_csv('submission_lgb.csv', index=False)
print(sample_sub.head())

         id    bg+1:00
0  p01_8459   8.234375
1  p01_8460   8.351562
2  p01_8461   8.320312
3  p01_8462  11.929688
4  p01_8463   8.031250


In [22]:
# Set up and run the Optuna study
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize")
study.optimize(objective_cat, n_trials=3)
print("Best parameters:", study.best_params)

0:	learn: 3.0229356	test: 3.0170545	best: 3.0170545 (0)	total: 4.7s	remaining: 1h 48m 29s
100:	learn: 1.9370012	test: 1.9666150	best: 1.9666150 (100)	total: 14.3s	remaining: 3m 1s
200:	learn: 1.7967225	test: 1.8863536	best: 1.8863536 (200)	total: 24s	remaining: 2m 21s
300:	learn: 1.6866723	test: 1.8352751	best: 1.8352751 (300)	total: 33.8s	remaining: 2m 1s
400:	learn: 1.5978030	test: 1.7955089	best: 1.7955089 (400)	total: 43.4s	remaining: 1m 46s
500:	learn: 1.5189096	test: 1.7651372	best: 1.7651372 (500)	total: 53.2s	remaining: 1m 33s
600:	learn: 1.4461044	test: 1.7355301	best: 1.7355301 (600)	total: 1m 2s	remaining: 1m 22s
700:	learn: 1.3818745	test: 1.7117340	best: 1.7117340 (700)	total: 1m 12s	remaining: 1m 11s
800:	learn: 1.3232171	test: 1.6915938	best: 1.6915938 (800)	total: 1m 22s	remaining: 1m
900:	learn: 1.2721699	test: 1.6753760	best: 1.6753760 (900)	total: 1m 32s	remaining: 49.7s
1000:	learn: 1.2207915	test: 1.6560842	best: 1.6560842 (1000)	total: 1m 42s	remaining: 39.2s
1100

In [23]:
sample_sub = pd.read_csv("/kaggle/input/brist1d/sample_submission.csv")
sample_sub['bg+1:00'] = cat_test.astype(np.float16)
sample_sub['id'] = test_id
sample_sub.drop(1)
sample_sub.to_csv('submission_cat.csv', index=False)
print(sample_sub.head())

         id    bg+1:00
0  p01_8459   8.117188
1  p01_8460   7.953125
2  p01_8461   8.289062
3  p01_8462  12.226562
4  p01_8463   8.062500


In [24]:
def blend_predictions(weights, oof_preds):
    """
    Blends predictions using the provided weights.
    
    Parameters:
    - weights (list or np.array): Weights for each model's predictions.
    - oof_preds (list of np.arrays): List of OOF predictions from different models.
    
    Returns:
    - np.array: Blended predictions.
    """
    blended_pred = np.zeros_like(oof_preds[0])
    for weight, oof in zip(weights, oof_preds):
        blended_pred += weight * oof
    return blended_pred


def evaluate_blend(weights, oof_preds, y_true):
    """
    Evaluates the blended predictions using a specified metric.
    
    Parameters:
    - weights (list or np.array): Weights for each model's predictions.
    - oof_preds (list of np.arrays): List of OOF predictions from different models.
    - y_true (np.array): True labels for the validation set.
    
    Returns:
    - float: RMSE score of the blended predictions.
    """
    blended_pred = blend_predictions(weights, oof_preds)
    score = np.sqrt(mean_squared_error(y_true, blended_pred))
    return score

def hill_climb_blend(oof_preds, y_true, initial_weights=None, step_size=0.05, max_iterations=100):
    """
    Performs hill climbing to find optimal weights for blending model predictions.
    
    Parameters:
    - oof_preds (list of np.arrays): List of OOF predictions from different models.
    - y_true (np.array): True labels for the validation set.
    - initial_weights (list or np.array, optional): Initial weights for each model.
    - step_size (float): Size of weight adjustments.
    - max_iterations (int): Maximum number of iterations for hill climbing.
    
    Returns:
    - np.array: Optimal weights for blending.
    - float: Best RMSE score achieved.
    """
    n_models = len(oof_preds)
    
    # Initialize weights
    if initial_weights is None:
        weights = np.array([1.0 / n_models] * n_models)  # Equal weights initially
    else:
        weights = np.array(initial_weights)
    
    # Normalize weights to sum to 1
    weights = weights / weights.sum()
    
    best_score = evaluate_blend(weights, oof_preds, y_true)
    best_weights = weights.copy()
    
    print(f"Initial weights: {best_weights}, Initial RMSE: {best_score:.5f}")
    
    for iteration in range(max_iterations):
        improved = False
        
        # Try increasing and decreasing each weight by `step_size`
        for i in range(n_models):
            for adjustment in [-step_size, step_size]:
                new_weights = best_weights.copy()
                new_weights[i] += adjustment
                
                # Normalize weights to sum to 1
                new_weights = np.maximum(new_weights, 0)  # Ensure no negative weights
                new_weights /= new_weights.sum()
                
                # Evaluate new weights
                new_score = evaluate_blend(new_weights, oof_preds, y_true)
                
                # If the new score is better, update best weights and score
                if new_score < best_score:
                    best_score = new_score
                    best_weights = new_weights
                    improved = True
                    print(f"Iteration {iteration}, Improved weights: {best_weights}, RMSE: {best_score:.5f}")
        
        # Stop if no improvement is made
        if not improved:
            break
    
    print(f"Optimal weights: {best_weights}, Best RMSE: {best_score:.5f}")
    return best_weights, best_score

# Example OOF predictions from each model (as numpy arrays)

oof_preds = [xgb_oof, lgb_oof, cat_oof]

# True labels for the validation set
y_true = y 

# Find the optimal weights
optimal_weights, best_score = hill_climb_blend(oof_preds, y_true, step_size=0.01, max_iterations=100)

print(f"Final optimal weights: {optimal_weights}, Best RMSE achieved: {best_score:.5f}")

Initial weights: [0.33333333 0.33333333 0.33333333], Initial RMSE: 1.35659
Iteration 0, Improved weights: [0.32659933 0.33670034 0.33670034], RMSE: 1.35638
Iteration 0, Improved weights: [0.32336567 0.34326766 0.33336667], RMSE: 1.35609
Iteration 0, Improved weights: [0.32663199 0.34673501 0.326633  ], RMSE: 1.35598
Iteration 1, Improved weights: [0.31983029 0.35023738 0.32993232], RMSE: 1.35578
Iteration 1, Improved weights: [0.31666366 0.35667068 0.32666567], RMSE: 1.35552
Iteration 1, Improved weights: [0.31986228 0.36027341 0.31986431], RMSE: 1.35543
Iteration 2, Improved weights: [0.3129922  0.36391254 0.32309526], RMSE: 1.35524
Iteration 2, Improved weights: [0.30989327 0.37021043 0.3198963 ], RMSE: 1.35500
Iteration 2, Improved weights: [0.3130235  0.37394993 0.31302656], RMSE: 1.35491
Iteration 3, Improved weights: [0.30608435 0.3777272  0.31618845], RMSE: 1.35473
Iteration 3, Improved weights: [0.30305381 0.38388832 0.31305787], RMSE: 1.35453
Iteration 3, Improved weights: [0.

In [25]:
ensemble_test = xgb_test *optimal_weights[0] + optimal_weights[1] *lgb_test + optimal_weights[2]*cat_test

In [26]:
sample_sub = pd.read_csv("/kaggle/input/brist1d/sample_submission.csv")
sample_sub['bg+1:00'] = ensemble_test.astype(np.float16)
sample_sub['id'] = test_id
sample_sub.drop(1)
sample_sub.to_csv('submission_ensemble.csv', index=False)
print(sample_sub.head())

         id    bg+1:00
0  p01_8459   8.273438
1  p01_8460   8.234375
2  p01_8461   8.328125
3  p01_8462  11.960938
4  p01_8463   7.886719
